# Pandas Methods & Indexing — Case Lab

Bu notebook bir **Pandas fundamentals** pratiğidir:  
- `Series` kavramını ve sık kullanılan **methods**’ları (örn. `.mean()`, `.sum()`, `.value_counts()`)  
- **parameter/argument** mantığını (örn. `numeric_only=True`)  
- `DataFrame` üzerinde **indexing/selection** (özellikle `.loc` ve `.iloc`)  
bir **mini-case** üzerinden öğretecek.

## Hedef
Ham bir CSV dosyasını yükleyip:
1) Veriyi hızlıca “tanımak” (inspection)  
2) Doğru method’larla özetlemek (summary)  
3) `.loc/.iloc` ile kontrollü seçimler yapmak  
4) Basit ama gerçekçi 2–3 **insight** çıkarmak

In [1]:
import pandas as pd

# Notebook çıktılarında kolonların daha rahat görünmesi için (opsiyonel)
pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 120)

In [2]:
df_raw = pd.read_csv("../data/Lending-company.csv")
# Data manipülasyonu olmasa bile bir refleks olmalı!
# Ham veri kopyası
df = df_raw.copy()
df.head ()

,LoanID,StringID,Product,CustomerGender,Location,Region,TotalPrice,StartDate,Deposit,DailyRate,TotalDaysYr,AmtPaid36,AmtPaid60,AmtPaid360,LoanStatus
0,1,LoanID_1,Product B,Female,Location 3,Region 2,17600.0,04/07/2018,2200,45,365,3221,4166,14621,Active
1,2,LoanID_2,Product D,Female,Location 6,Region 6,NaN,02/01/2019,2200,45,365,3161,4096,16041,Active
2,3,LoanID_3,Product B,Male,Location 8,Region 3,16600.0,08/12/2016,1000,45,365,2260,3205,16340,NaN
3,4,LoanID_4,Product A,Male,Location 26,Region 2,17600.0,NaN,2200,45,365,3141,4166,16321,Active
4,5,LoanID_5,Product B,Female,Location 34,Region 3,21250.0,28/10/2017,2200,55,365,3570,4745,14720,Active


## Attributes vs Methods

Pandas’ta:

- **Attributes**: Parantez yok → hızlı bilgi
  - `df.shape`, `df.columns`, `df.dtypes`
- **Methods**: Parantez var → işlem/hesap
  - `df.info()`, `df.describe()`, `df.mean()`

> Kural: `()` görüyorsan bir iş yaptırıyorsun.

In [3]:
df.shape, df.columns

((1043, 15),
 Index(['LoanID', 'StringID', 'Product', 'CustomerGender', 'Location', 'Region', 'TotalPrice', 'StartDate', 'Deposit',
        'DailyRate', 'TotalDaysYr', 'AmtPaid36', 'AmtPaid60', 'AmtPaid360', 'LoanStatus'],
       dtype='object'))

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1043 entries, 0 to 1042
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   LoanID          1043 non-null   int64  
 1   StringID        1043 non-null   object 
 2   Product         1043 non-null   object 
 3   CustomerGender  1043 non-null   object 
 4   Location        1043 non-null   object 
 5   Region          1042 non-null   object 
 6   TotalPrice      1018 non-null   float64
 7   StartDate       1042 non-null   object 
 8   Deposit         1043 non-null   int64  
 9   DailyRate       1043 non-null   int64  
 10  TotalDaysYr     1043 non-null   int64  
 11  AmtPaid36       1043 non-null   int64  
 12  AmtPaid60       1043 non-null   int64  
 13  AmtPaid360      1043 non-null   int64  
 14  LoanStatus      1006 non-null   object 
dtypes: float64(1), int64(7), object(7)
memory usage: 122.4+ KB


In [5]:
# Sayısal kolonları ayırt etme

df.select_dtypes(include="number").columns

Index(['LoanID', 'TotalPrice', 'Deposit', 'DailyRate', 'TotalDaysYr', 'AmtPaid36', 'AmtPaid60', 'AmtPaid360'], dtype='object')

In [6]:
df.describe()

,LoanID,TotalPrice,Deposit,DailyRate,TotalDaysYr,AmtPaid36,AmtPaid60,AmtPaid360
count,1043.000000,1018.000000,1043.000000,1043.000000,1043.0,1043.000000,1043.000000,1043.000000
mean,522.000000,19562.843811,2528.092042,50.834132,365.0,3901.857143,5214.393097,17122.131352
std,301.232468,5955.588065,1504.467724,11.567390,0.0,2537.597667,2850.327133,5910.916736
min,1.000000,13475.000000,1000.000000,30.000000,365.0,-2770.000000,-2945.000000,650.000000
25%,261.500000,16300.000000,2200.000000,45.000000,365.0,2840.500000,3822.000000,15560.500000
50%,522.000000,17600.000000,2200.000000,45.000000,365.0,3420.000000,4742.000000,16617.000000
75%,782.500000,20950.000000,2200.000000,55.000000,365.0,5075.500000,6393.000000,19977.500000
max,1043.000000,70225.000000,8000.000000,150.000000,365.0,18851.000000,22143.000000,65001.000000


In [7]:
#Parametre kavramı

df.mean(numeric_only=True)

LoanID           522.000000
TotalPrice     19562.843811
Deposit         2528.092042
DailyRate         50.834132
TotalDaysYr      365.000000
AmtPaid36       3901.857143
AmtPaid60       5214.393097
AmtPaid360     17122.131352
dtype: float64

## Numeric vs Categorical Columns = Neyi Özetleyeceğiz?

Özet istatistik üretmeden önce şu soruyu sormalıyız:

> Hangi kolonlar gerçekten sayısal?

Çünkü `.mean()` veya `.sum()` gibi method’lar
yalnızca sayısal veriler için anlamlıdır.

Bu yüzden önce veri tiplerini bilinçli şekilde ayırt edeceğiz.

In [8]:
df.select_dtypes(include="number").columns

Index(['LoanID', 'TotalPrice', 'Deposit', 'DailyRate', 'TotalDaysYr', 'AmtPaid36', 'AmtPaid60', 'AmtPaid360'], dtype='object')

Bu çıktı bize şunu gösteriyor:

- Hangi kolonlar üzerinde aritmetik işlemler yapabiliriz?
- Hangi kolonlar kategorik (object) olduğu için farklı analiz gerektirir?

Profesyonel veri analizinde ilk refleks:
"Önce veri tipini tanı."

In [9]:
df.describe()

,LoanID,TotalPrice,Deposit,DailyRate,TotalDaysYr,AmtPaid36,AmtPaid60,AmtPaid360
count,1043.000000,1018.000000,1043.000000,1043.000000,1043.0,1043.000000,1043.000000,1043.000000
mean,522.000000,19562.843811,2528.092042,50.834132,365.0,3901.857143,5214.393097,17122.131352
std,301.232468,5955.588065,1504.467724,11.567390,0.0,2537.597667,2850.327133,5910.916736
min,1.000000,13475.000000,1000.000000,30.000000,365.0,-2770.000000,-2945.000000,650.000000
25%,261.500000,16300.000000,2200.000000,45.000000,365.0,2840.500000,3822.000000,15560.500000
50%,522.000000,17600.000000,2200.000000,45.000000,365.0,3420.000000,4742.000000,16617.000000
75%,782.500000,20950.000000,2200.000000,55.000000,365.0,5075.500000,6393.000000,19977.500000
max,1043.000000,70225.000000,8000.000000,150.000000,365.0,18851.000000,22143.000000,65001.000000


`.describe()` bize şu bilgileri verir:

- count → Kaç non-null gözlem var?
- mean → Ortalama
- std → Standart sapma
- min / max → Aralık
- 25%, 50%, 75% → Çeyrekler

Burada kritik soru:

> TotalPrice kolonunda neden count 1043 değil?

Bu bizi eksik veri konusuna götürür.

## .describe() Çıktısı Üzerine Veri Kalitesi ve Analizi

`.describe()` fonksiyonu bize sayısal değişkenler için temel istatistikleri verir:

- **count** → Non-null gözlem sayısı  
- **mean** → Ortalama  
- **std** → Standart sapma  
- **min / max** → Aralık  
- **25%, 50%, 75%** → Çeyrekler (quantile)

---

### Eksik Veri Tespiti

- Veri setinde toplam **1043 gözlem** bulunmaktadır.
- `TotalPrice` kolonunda **1018 gözlem** vardır.
- Bu durum ilgili değişkende **25 adet eksik değer (NaN)** olduğunu göstermektedir.

 Bu durum veri kalitesi açısından not edilmelidir ve analiz öncesinde eksik değer stratejisi belirlenmelidir.

---

### Negatif Ödeme Değerleri (Kritik Gözlem)

Aşağıdaki değişkenlerde negatif minimum değerler gözlenmiştir:

- `AmtPaid36` → min = -2770  
- `AmtPaid60` → min = -2945  

 Finansal bir veri setinde negatif ödeme değerleri:

- İade (refund)
- Chargeback
- Veri giriş hatası
- Muhasebe düzeltme kaydı

gibi durumları işaret edebilir.

Bu değerler analiz öncesinde doğrulanmalı ve anlamı netleştirilmelidir.

---

### Sabit Kolon Problemi

`TotalDaysYr` değişkeni için:

- mean = 365  
- std = 0  
- min = 365  
- max = 365  

Bu durum tüm gözlemlerin 365 olduğunu göstermektedir.

  Bu değişken varyasyon içermediği için:
- Modelleme açısından bilgi taşımaz.
- Analitik katkısı düşüktür.
- Feature olarak çıkarılması değerlendirilebilir.

---

### Deposit Değişkeni Dağılımı

`Deposit` değişkeninde:

- 25%, 50% ve 75% değerlerinin 2200 olması,
- Ödemelerin büyük ölçüde bu değerde yoğunlaştığını göstermektedir.

 Bu durum değişkenin ayrık (discrete) ve kümelenmiş bir dağılıma sahip olduğunu düşündürmektedir.

---

### Genel Analitik Değerlendirme

- Veri seti genel olarak tutarlı görünmektedir.
- Eksik veri sınırlıdır (yalnızca `TotalPrice` değişkeninde).
- Negatif ödeme değerleri dikkatle incelenmelidir.
- Sabit kolonlar analiz gücünü düşürebilir.
- Deposit dağılımı homojen değildir.

Bu gözlemler veri temizleme ve ileri analiz aşamalarında dikkate alınmalıdır.

In [10]:
df.mean(numeric_only=True)

LoanID           522.000000
TotalPrice     19562.843811
Deposit         2528.092042
DailyRate         50.834132
TotalDaysYr      365.000000
AmtPaid36       3901.857143
AmtPaid60       5214.393097
AmtPaid360     17122.131352
dtype: float64

Burada önemli kavram:

- `.mean()` bir method
- `numeric_only=True` bir parametre

Parametreler method’un nasıl davranacağını değiştirir.

Eğer numeric_only belirtmezsek,
Pandas bazı durumlarda hata verebilir veya object kolonları dışarıda bırakır.

Advanced düşünce:

> Method çağırmak yetmez, nasıl çağırdığını bilmek gerekir.

**Ortalama TotalPrice**

In [11]:
df["TotalPrice"].mean()

19562.84381139489

Insight 1:

Ortalama kredi tutarı nedir?

Bu bize dataset'in genel büyüklüğü hakkında fikir verir.

**LoanStatus Dağılımı**

In [12]:
df["LoanStatus"].value_counts()

LoanStatus
Finished Payment    534
Active              460
Blocked              12
Name: count, dtype: int64

`.value_counts()` kategorik değişkenler için en güçlü method'lardan biridir.

Bu dağılım bize:

- Kaç kredi aktif?
- Kaç kredi kapalı?
- Kaç kredi gecikmede?

sorularına hızlı cevap verir.

**Eksik Veri Analizi**

In [13]:
df.isna().sum()

LoanID             0
StringID           0
Product            0
CustomerGender     0
Location           0
Region             1
TotalPrice        25
StartDate          1
Deposit            0
DailyRate          0
TotalDaysYr        0
AmtPaid36          0
AmtPaid60          0
AmtPaid360         0
LoanStatus        37
dtype: int64

Bu tablo bize her kolon için eksik değer sayısını gösterir.

Advanced refleks:

Eksik veri varsa:

- Silmeli miyiz?
- Doldurmalı mıyuz?
- Yoksa analiz dışında mı bırakmalıyız?

Veri analizi sadece method çağırmak değil,
veri kalitesini sorgulamaktır.

## Scenario 1 Region 2 Kredileri

Şimdi belirli bir bölgeyi filtreleyelim.

`.loc` ile koşullu seçim yapabiliriz.

Mantık:

df.loc[koşul, kolonlar]

In [14]:
df.loc[df["Region"] == "Region 2"].head()

,LoanID,StringID,Product,CustomerGender,Location,Region,TotalPrice,StartDate,Deposit,DailyRate,TotalDaysYr,AmtPaid36,AmtPaid60,AmtPaid360,LoanStatus
0,1,LoanID_1,Product B,Female,Location 3,Region 2,17600.0,04/07/2018,2200,45,365,3221,4166,14621,Active
3,4,LoanID_4,Product A,Male,Location 26,Region 2,17600.0,NaN,2200,45,365,3141,4166,16321,Active
15,16,LoanID_16,Product B,Male,Location 44,Region 2,17600.0,31/05/2018,2200,45,365,3221,4726,16001,Active
23,24,LoanID_24,Product C,NotSpecified,Location 91,Region 2,16600.0,21/12/2016,1000,45,365,2340,4485,15520,Active
25,26,LoanID_26,Product A,Male,Location 75,Region 2,23250.0,11/09/2019,5000,55,365,6400,6525,15800,Active


Burada:

- `df["Region"] == "Region 2"` → Boolean seri üretir
- `.loc` bu True olan satırları getirir

Advanced düşünce:

`.loc` label-based çalışır.
Yani kolon adlarını kullanır.

**Fiyat Üzeri Filtre**

In [15]:
df.loc[df["TotalPrice"] > 20000].head()

,LoanID,StringID,Product,CustomerGender,Location,Region,TotalPrice,StartDate,Deposit,DailyRate,TotalDaysYr,AmtPaid36,AmtPaid60,AmtPaid360,LoanStatus
4,5,LoanID_5,Product B,Female,Location 34,Region 3,21250.0,28/10/2017,2200,55,365,3570,4745,14720,Active
6,7,LoanID_7,Product A,Male,Location 25,NaN,21250.0,04/07/2020,2200,55,365,1951,3176,18701,Active
8,9,LoanID_9,Product A,Male,Location 156,Region 6,23250.0,03/09/2019,5000,55,365,5850,7375,21250,NaN
9,10,LoanID_10,Product C,Male,Location 21,Region 9,21250.0,25/07/2020,2200,55,365,2051,3176,18351,Active
11,12,LoanID_12,Product D,Female,Location 25,Region 6,21250.0,29/03/2017,2200,55,365,3190,4115,11790,Active


Bu, veri analizi dünyasında en sık kullanılan kalıptır:

df.loc[df["kolon"] > değer]

Bu pattern’i öğrenmek Pandas’ın %40’ını öğrenmek demektir.

**Çoklu Koşul**

In [16]:
df.loc[
    (df["Region"] == "Region 2") & 
    (df["TotalPrice"] > 20000)
].head()

,LoanID,StringID,Product,CustomerGender,Location,Region,TotalPrice,StartDate,Deposit,DailyRate,TotalDaysYr,AmtPaid36,AmtPaid60,AmtPaid360,LoanStatus
25,26,LoanID_26,Product A,Male,Location 75,Region 2,23250.0,11/09/2019,5000,55,365,6400,6525,15800,Active
42,43,LoanID_43,Product A,Female,Location 42,Region 2,20250.0,17/10/2019,1000,55,365,-200,125,9918,Active
75,76,LoanID_76,Product A,Male,Location 11,Region 2,23250.0,11/09/2019,5000,55,365,5450,6725,18400,Active
76,77,LoanID_77,Product A,Male,Location 75,Region 2,20250.0,31/01/2020,1000,55,365,900,1775,13650,Active
80,81,LoanID_81,Product A,NotSpecified,Location 44,Region 2,20250.0,28/11/2019,1000,55,365,500,1175,9450,Active


Birden fazla koşul:

- `&` → and
- `|` → or

Parantez zorunludur.

**iloc ile Konumsal Seçim**

In [17]:
df.iloc[0]

LoanID                     1
StringID            LoanID_1
Product            Product B
CustomerGender        Female
Location          Location 3
Region              Region 2
TotalPrice           17600.0
StartDate         04/07/2018
Deposit                 2200
DailyRate                 45
TotalDaysYr              365
AmtPaid36               3221
AmtPaid60               4166
AmtPaid360             14621
LoanStatus            Active
Name: 0, dtype: object

In [18]:
df.iloc[0:5, 0:3]

,LoanID,StringID,Product
0,1,LoanID_1,Product B
1,2,LoanID_2,Product D
2,3,LoanID_3,Product B
3,4,LoanID_4,Product A
4,5,LoanID_5,Product B


- `.loc` → label (isim)
- `.iloc` → integer position
- 
Model eğitirken genellikle `.iloc` kullanılır.

In [ ]:
df["Region"].unique()
df["Location"].unique()

## Insight

Region 2'deki yüksek fiyatlı krediler,
genel ortalamadan sapma gösteriyor mu?

Bu tür sorular,
veri analizinin başlangıç refleksidir.

In [20]:
df.groupby("Region")["TotalPrice"].mean().sort_values(ascending=False)

Region
Region 3     20345.625000
Region 14    19950.000000
Region 4     19872.177419
Region 1     19770.200000
Region 10    19707.142857
Region 15    19525.000000
Region 6     19439.375000
Region 17    19400.000000
Region 2     19308.016304
Region 12    19275.000000
Region 7     19127.272727
Region 9     18391.666667
Region 8     18050.000000
Region 5     17908.333333
Region 11    17600.000000
Region 16    16950.000000
Region 13    16525.000000
Region 18    16300.000000
Name: TotalPrice, dtype: float64

### Insight Interpretation

- Region 3 en yüksek ortalama kredi tutarına sahip (~20,345)
- Region 18 en düşük ortalamaya sahip (~16,300)
- Region 2, sandığımız kadar yüksek değil (9. sırada)

Bu bize şunu gösteriyor:

Tek bir filtre ile yaptığımız gözlem,
genel dağılım içinde bağlamlandırılmalıdır.

Veri analizinde bağlam (context) her şeydir.